# Parsing PDFs Homework

With the power of pdfminer, pytesseract, Camelot, and Tika, let's analyze some documents!

> If at any point you think, **"I'm close enough, I'd just edit the rest of it in Excel"**: that's fine! Just make a note of it.

## A trick to use again and again

### Approach 1

Before we get started: when you want to take the first row of your data and set it as the header, use this trick.

In [1]:
import pandas as pd

In [2]:
df = pd.DataFrame([
    [ 'fruit name', 'likes' ],
    [ 'apple', 15 ],
    [ 'carrot', 3 ],
    [ 'sweet potato', 45 ],
    [ 'peach', 12 ],
])
df

,0,1
0,fruit name,likes
1,apple,15
2,carrot,3
3,sweet potato,45
4,peach,12


In [3]:
# Set the first column as the columns
df.columns = df.loc[0]

# Drop the first row
df = df.drop(0)

df

,fruit name,likes
1,apple,15
2,carrot,3
3,sweet potato,45
4,peach,12


🚀 Done!

### Approach 2

Another alternative is to use `.rename` on your columns and just filter out the columns you aren't interested in. This can be useful if the column name shows up multiple times in your data for some reason or another.

In [5]:
# Starting with the same-ish data...
df = pd.DataFrame([
    [ 'fruit name', 'likes' ],
    [ 'apple', 15 ],
    [ 'carrot', 3 ],
    [ 'fruit name', 'likes' ],
    [ 'sweet potato', 45 ],
    [ 'peach', 12 ],
])
df

,0,1
0,fruit name,likes
1,apple,15
2,carrot,3
3,fruit name,likes
4,sweet potato,45
5,peach,12


In [6]:
df = df.rename(columns={
    0: 'fruit name',
    1: 'likes'
})
df = df[df['fruit name'] != 'fruit name']
df

,fruit name,likes
1,apple,15
2,carrot,3
4,sweet potato,45
5,peach,12


🚀 Done!

### Useful tips about coordinates

If you want to grab only a section of the page [Kull](https://jsoma.github.io/kull/#/) might be helpful in finding the coordinates.

> **Alternatively** run `%matplotlib notebook` in a cell. Afterwards, every time you use something like `camelot.plot(tables[0]).show()` it will get you nice zoomable, hoverable versions that include `x` and `y` coordinates as you move your mouse.

Coordinates are given as `"left_x,top_y,right_x,bottom_y"` with `(0,0)` being in the bottom left-hand corner.

Note that all coordinates are strings, for some reason. It won't be `[1, 2, 3, 4]` it will be `['1,2,3,4']`

# Camelot questions

The largest part of this assignment is **mostly Camelot work**. As tabular data is usually the most typical data you'll be working with, it's what I'm giving you!

It will probably be helpful to read through [Camelot's advanced usage tips](https://camelot-py.readthedocs.io/en/master/user/advanced.html), along with the notebook I posted in the homework assignment.

## Prison Inmates

Working from [InmateList.pdf](InmateList.pdf), save a CSV file that includes every inmate.

* Make sure your rows are *all data*, and you don't have any people named "Inmate Name."


In [4]:
import camelot

In [78]:
tables = camelot.read_pdf("InmateList.pdf", flavor="stream")
tables

<TableList n=1>

In [79]:
tables[0].df.head()

,0,1,2,3,4,5
0,,,,Erie County Sheriff's Office,,
1,,,,Inmate Roster,,
2,ICN #,Inmate Name,,Facility,Booking Date,
3,70693,"ABDALLAH, MICHAEL",,ECHC,04/30/2021,
4,152645,"ABDI, ABDI",,ECCF,06/20/2021,


In [80]:
tables = camelot.read_pdf("InmateList.pdf", flavor="stream", pages="all")
tables[15].df.tail()

,0,1,2,3
37,18303,"ZANGHI, SANTINO J",ECHC,05/24/2021
38,155109,"ZARCONE, THOMAS E",ECHC,06/27/2021
39,63815,"ZIELINSKI, LARRY",ECHC,07/06/2021
40,20740,"ZIELINSKI, LAWRENCE T",ECCF,11/04/2020
41,155732,"ZUKIC, TRAVIS",ECCF,03/01/2020


In [16]:
import pandas as pd

In [83]:
dfs = [table.df for table in tables]
df = pd.concat(dfs, ignore_index=True)
df.tail()
df.shape

(747, 6)

In [87]:
df.columns = df.loc[2]
df
# Will clean up the column names in spreadsheet

2,ICN #,Inmate Name,,Facility,Booking Date,
0,,,,Erie County Sheriff's Office,,
1,,,,Inmate Roster,,
2,ICN #,Inmate Name,,Facility,Booking Date,
3,70693,"ABDALLAH, MICHAEL",,ECHC,04/30/2021,
4,152645,"ABDI, ABDI",,ECCF,06/20/2021,
...,...,...,...,...,...,...
742,18303,"ZANGHI, SANTINO J",ECHC,05/24/2021,NaN,NaN
743,155109,"ZARCONE, THOMAS E",ECHC,06/27/2021,NaN,NaN
744,63815,"ZIELINSKI, LARRY",ECHC,07/06/2021,NaN,NaN
745,20740,"ZIELINSKI, LAWRENCE T",ECCF,11/04/2020,NaN,NaN


In [88]:
df = df[df['Inmate Name'] != 'Inmate Name']

In [89]:
df.shape

(731, 6)

In [92]:
df = df[df['Facility'] != "Erie County Sheriff's Office"]
df.shape

(716, 6)

In [94]:
df = df[df['Facility'] != "Inmate Roster"]
df.shape

(701, 6)

In [96]:
df = df[df['ICN #'] != "Created On:"]
df.shape

(686, 6)

In [98]:
df.tail()

2,ICN #,Inmate Name,,Facility,Booking Date,
742,18303,"ZANGHI, SANTINO J",ECHC,05/24/2021,NaN,NaN
743,155109,"ZARCONE, THOMAS E",ECHC,06/27/2021,NaN,NaN
744,63815,"ZIELINSKI, LARRY",ECHC,07/06/2021,NaN,NaN
745,20740,"ZIELINSKI, LAWRENCE T",ECCF,11/04/2020,NaN,NaN
746,155732,"ZUKIC, TRAVIS",ECCF,03/01/2020,NaN,NaN


In [103]:
df = df.reset_index(drop=True)
df

2,ICN #,Inmate Name,,Facility,Booking Date,
0,70693,"ABDALLAH, MICHAEL",,ECHC,04/30/2021,
1,152645,"ABDI, ABDI",,ECCF,06/20/2021,
2,144666,"ABDULLAH, DHAFIR",,ECCF,06/17/2021,
3,156374,"ACEVEDO, CARLOS",,ECHC,06/06/2021,
4,57243,"ACKER, RAYMOND P",,ECCF,11/02/2020,
...,...,...,...,...,...,...
681,18303,"ZANGHI, SANTINO J",ECHC,05/24/2021,NaN,NaN
682,155109,"ZARCONE, THOMAS E",ECHC,06/27/2021,NaN,NaN
683,63815,"ZIELINSKI, LARRY",ECHC,07/06/2021,NaN,NaN
684,20740,"ZIELINSKI, LAWRENCE T",ECCF,11/04/2020,NaN,NaN


In [104]:
df.to_csv("InmateList.csv", index=False)
# To remove rows 260 and 261 from the table and move some rows for "facility" and "dates" to the right columns

## WHO resolutions

Using [A74_R13-en.pdf](A74_R13-en.pdf), what ten member countries are given the highest assessments?

* You might need to have two separate queries, and combine the results: that last page is pretty awful!
* Always rename your columns
* Double-check that your sorting looks right......
* You can still get the answer even without perfectly clean data

In [226]:
tables = camelot.read_pdf("A74_R13-en.pdf", flavor="stream", pages="all")
tables 

<TableList n=6>

In [227]:
dfs = [table.df for table in tables]
df = pd.concat(dfs, ignore_index=True)
df.shape

(225, 5)

In [228]:
df.head()

,0,1,2,3,4
0,Members and,WHO scale,NaN,NaN,NaN
1,Associate Members,for 2022–2023,NaN,NaN,NaN
2,,%,NaN,NaN,NaN
3,Afghanistan,0.0070,NaN,NaN,NaN
4,Albania,0.0080,NaN,NaN,NaN


In [229]:
df.tail()

,0,1,2,3,4
220,,Zimbabwe,,0.0050,
221,,TOTAL,,100.000,
222,,,,,"Seventh plenary meeting, 31 May 2021"
223,,,,,A74/VR/7
224,6,,= = =,,


In [230]:
# redo with two queries 
tables_1 = camelot.read_pdf("A74_R13-en.pdf", flavor="stream", pages="1-5")
tables_2 = camelot.read_pdf("A74_R13-en.pdf", flavor="stream", pages="6")

In [281]:
dfs = [table.df for table in tables_1]
df = pd.concat(dfs, ignore_index=True)
df

,0,1
0,Members and,WHO scale
1,Associate Members,for 2022–2023
2,,%
3,Afghanistan,0.0070
4,Albania,0.0080
...,...,...
210,Uzbekistan,0.0320
211,Vanuatu,0.0010
212,Venezuela (Bolivarian Republic of),0.7280
213,Viet Nam,0.0770


In [282]:
df = df.rename(columns={
    0: 'Members',
    1: 'WHO scale'
})

In [283]:
df = df[df['Members'] != 'Members and']
df = df[df['Members'] != 'Associate Members']
df = df[df['WHO scale'] != '%']
df = df[df['Members'] != 'Democratic People’s Republic of']
df = df[df['Members'] != 'United Kingdom of Great Britain and']
df.rename(index={'Northern Ireland':'UK'},inplace=True)
df.shape

(198, 2)

In [256]:
# Tried cleaning the rows with places that say"not a member of the United Nations" using regex but didn't make it work. 
# But these places' WHO scale values are all very low so will ignore them. 
# Will similarly ignore the 2 countries on last page with low values

In [289]:
# Top 10 countries with highest assessments

df=df.sort_values('WHO scale', ascending = False)
df.head(10)

,Members,WHO scale
97,Japan,8.5645
77,Germany,6.0904
206,Northern Ireland,4.5673
70,France,4.4273
95,Italy,3.3072
208,United States of America,22.0000
29,Brazil,2.9482
37,Canada,2.7342
160,Russian Federation,2.4052
157,Republic of Korea,2.2671


## The Avengers

Using [THE_AVENGERS.pdf](THE_AVENGERS.pdf), approximately how many lines does Captain America have as compared to Thor and Iron Man?

* Character names only: we're only counting `IRON MAN` as Iron Man, not `TONY`.
* Your new best friend might be `\n`
* Look up `.count` for strings

In [291]:
from pdfminer.high_level import extract_text
text = extract_text('THE_AVENGERS.pdf')

In [293]:
print(text)

Marvel’s THE AVENGERS

Written By

Joss Whedon

Story By
Zak Penn and Joss Whedon

Based on the characters appearing in the comic books
Published by MARVEL Comics

Transcribed to PDF using:
Celtx Studio | 2012

Composed In Celtx Studio
With the help of
SiOmniaFicta’s Script
on fanfiction.net

ENTERTAINMENT PURPOSES ONLY

1

EXT. UNKNOWN AREA OF SPACE

Space, a floating staircase among the rocks...THE OTHER
speaks to a DISEMBODIED FIGURE in a chair.

THE OTHER (V.O)

The Tesseract has awakened. It is
on a little world, a human world.
They would wield its power.

THE OTHER hands LOKI a scepter with a glowing blue gem...

THE OTHER (V.O)
But our ally knows its workings as
they never will. He is ready to
lead. And our force...

The CHITAURI put on their face mask and get ready.

THE OTHER (V.O)

...our Chitauri, will follow. The
world will be his...the universe,
yours. And the humans, what can
they do but burn?

A 3d model of the Tesseract, we move inside it slowly

EXT. MOUNTAIN RANGE - 

In [303]:
# No. of lines for Captain America
text.count("\nCAPTAIN AMERICA")

39

In [304]:
# No. of lines for Thor
text.count("\nTHOR")

47

In [305]:
# No. of lines for Iron Man
text.count("\nIRON MAN")

21

## COVID data

Using [covidweekly2721.pdf](covidweekly2721.pdf), what's the total number of tests performed in Minnesota? Use the Laboratory Test Rates by County of Residence chart.

* You COULD pull both tables separately OR you could pull them both at once and split them in pandas.
* Remember you can do things like `df[['name','age']]` to ask for multiple columns

In [414]:
tables = camelot.read_pdf("covidweekly2721.pdf", pages="6")
tables 

<TableList n=2>

In [415]:
df = tables[1].df
df

,0,1,2,3,4,5
0,County,Number of Tests,Cumulative Rate,County,Number of Tests,Cumulative Rate
1,Aitkin,"19,204","12,128",Martin,"39,876","19,974"
2,Anoka,"545,958","15,714",McLeod,"61,300","17,111"
3,Becker,"59,238","17,540",Meeker,"37,736","16,351"
4,Beltrami,"60,345","13,085",Mille Lacs,"44,688","17,369"
5,Benton,"77,865\n9,953","26,976",Morrison,"59,179","17,961"
6,Big Stone,"13,220","26,356",Mower,"79,446","20,061"
7,Blue Earth,"136,895","20,641",Murray,"15,175","18,167"
8,Brown,"55,709","22,097",Nicollet,"66,163","19,585"
9,Carlton,"79,882","22,477",Nobles,"30,979","14,185"


In [416]:
df.columns = df.loc[0]

In [417]:
df = df.drop(0)

In [418]:
df

,County,Number of Tests,Cumulative Rate,County,Number of Tests,Cumulative Rate
1,Aitkin,"19,204","12,128",Martin,"39,876","19,974"
2,Anoka,"545,958","15,714",McLeod,"61,300","17,111"
3,Becker,"59,238","17,540",Meeker,"37,736","16,351"
4,Beltrami,"60,345","13,085",Mille Lacs,"44,688","17,369"
5,Benton,"77,865\n9,953","26,976",Morrison,"59,179","17,961"
6,Big Stone,"13,220","26,356",Mower,"79,446","20,061"
7,Blue Earth,"136,895","20,641",Murray,"15,175","18,167"
8,Brown,"55,709","22,097",Nicollet,"66,163","19,585"
9,Carlton,"79,882","22,477",Nobles,"30,979","14,185"
10,Carver,"154,305","15,367",Norman,"11,471","17,489"


In [383]:
# I'll just save the above df to csv to merge the same columns together. 
# Spent a day looking for ways to split this damn table in the middle but didn't manage to find the right codes. Using Kull to get the table by coordinates gave a messier output.

## Theme Parks

Using [2019-Theme-Index-web-1.pdf](2019-Theme-Index-web-1.pdf), save a CSV of the top 10 theme park groups worldwide.

* You can clean the results or you can restrict the area the table is pulled from, up to you

In [419]:
table = camelot.read_pdf('2019-Theme-Index-web-1.pdf', pages="11", flavor='stream', table_areas=['26,469,380,280'])
table[0].df.head(13)

,0,1,2,3,4
0,1,WALT DISNEY ATTRACTIONS,-0.8%,"155,991,000","157,311,000"
1,2,MERLIN ENTERTAINMENTS GROUP,0.9%,"67,000,000","66,400,000*"
2,3,OCT PARKS CHINA,9.4%,"53,970,000","49,350,000"
3,4,UNIVERSAL PARKS AND RESORTS,2.3%,"51,243,000","50,068,000"
4,5,FANTAWILD GROUP,19.8%,"50,393,000","42,074,000"
5,6,CHIMELONG GROUP,8.9%,"37,018,000","34,007,000"
6,7,SIX FLAGS INC.,2.5%,"32,811,000","32,024,000"
7,8,CEDAR FAIR ENTERTAINMENT COMPANY,7.8%,"27,938,000","25,912,000"
8,9,SEAWORLD PARKS & ENTERTAINMENT,0.2%,"22,624,000","22,582,000"
9,10,PARQUES REUNIDOS,6.2%,"22,195,000","20,900,000"


In [421]:
df = table[0].df.head(13)
df

,0,1,2,3,4
0,1,WALT DISNEY ATTRACTIONS,-0.8%,"155,991,000","157,311,000"
1,2,MERLIN ENTERTAINMENTS GROUP,0.9%,"67,000,000","66,400,000*"
2,3,OCT PARKS CHINA,9.4%,"53,970,000","49,350,000"
3,4,UNIVERSAL PARKS AND RESORTS,2.3%,"51,243,000","50,068,000"
4,5,FANTAWILD GROUP,19.8%,"50,393,000","42,074,000"
5,6,CHIMELONG GROUP,8.9%,"37,018,000","34,007,000"
6,7,SIX FLAGS INC.,2.5%,"32,811,000","32,024,000"
7,8,CEDAR FAIR ENTERTAINMENT COMPANY,7.8%,"27,938,000","25,912,000"
8,9,SEAWORLD PARKS & ENTERTAINMENT,0.2%,"22,624,000","22,582,000"
9,10,PARQUES REUNIDOS,6.2%,"22,195,000","20,900,000"


In [426]:
df = df.rename(columns={
    0: 'rank',
    1: 'group_name',
    2: 'percent-change',
    3: 'attendance_2019',
    4: 'attendance_2020'
})

df.to_csv("theme_park.csv", index=False)

## Hunting licenses

Using [US_Fish_and_Wildlife_Service_2021.pdf](US_Fish_and_Wildlife_Service_2021.pdf) and [a CSV of state populations](http://goodcsv.com/geography/us-states-territories/), find the states with the highest per-capita hunting license holders.

In [610]:
tables = camelot.read_pdf("US_Fish_and_Wildlife_Service_2021.pdf")
tables[0].df
df1 = tables[0].df

In [611]:
df1.columns = df1.loc[0]
df1.head()

,State,Paid Hunting License \nHolders*,"Resident Hunting Licenses,\nTags, Permits and Stamps","Non-Resident Hunting \nLicenses,\nTags, Permits and Stamps","Total Hunting License, \nTags,Permits & Stamps**","Cost - Resident Hunting \nLicenses,\nTags, Permits and Stamps","Cost - Non-Resident Hunting \nLicenses,\nTags, Permits and Stamps",Gross Cost - Hunting \nLicenses
0,State,Paid Hunting License \nHolders*,"Resident Hunting Licenses,\nTags, Permits and ...","Non-Resident Hunting \nLicenses,\nTags, Permit...","Total Hunting License, \nTags,Permits & Stamps**","Cost - Resident Hunting \nLicenses,\nTags, Per...","Cost - Non-Resident Hunting \nLicenses,\nTags,...",Gross Cost - Hunting \nLicenses
1,AK,"93,559","423,501","59,235","482,736","$4,859,356","$9,046,715","$13,906,071"
2,AL,"452,400","601,683","45,397","647,080","$9,700,295","$6,715,734","$16,416,029"
3,AR,"343,300","349,098","150,728","499,826","$7,851,601","$11,271,653","$19,123,254"
4,AS,0,0,0,0,$0,$0,$0


In [612]:
df1 = df1.drop(0)

In [613]:
df1.head()

,State,Paid Hunting License \nHolders*,"Resident Hunting Licenses,\nTags, Permits and Stamps","Non-Resident Hunting \nLicenses,\nTags, Permits and Stamps","Total Hunting License, \nTags,Permits & Stamps**","Cost - Resident Hunting \nLicenses,\nTags, Permits and Stamps","Cost - Non-Resident Hunting \nLicenses,\nTags, Permits and Stamps",Gross Cost - Hunting \nLicenses
1,AK,"93,559","423,501","59,235","482,736","$4,859,356","$9,046,715","$13,906,071"
2,AL,"452,400","601,683","45,397","647,080","$9,700,295","$6,715,734","$16,416,029"
3,AR,"343,300","349,098","150,728","499,826","$7,851,601","$11,271,653","$19,123,254"
4,AS,0,0,0,0,$0,$0,$0
5,AZ,"302,383","464,607","88,708","553,315","$13,931,397","$5,968,169","$19,899,566"


In [614]:
# add encoding='latin1' to load the second file
df2 = pd.read_csv("us-states-territories.csv", encoding='latin1') 

In [615]:
df2.head()
df2.describe()

,Type,Name,Abbreviation,Capital,Population (2015),Population (2019),area (square miles)
count,65,65,56,55,4,52,65
unique,3,65,56,55,4,52,64
top,State,Alabama,AL,Montgomery,"57,400","4,903,185",3
freq,50,1,1,1,1,1,2


In [616]:
# Wonder why this code doesn't work in merging the two data sets 
# keeps getting keyError for "State". Only the DC row is merged
df1.merge(df2, left_on='State', right_on='Abbreviation')

,State,Paid Hunting License \nHolders*,"Resident Hunting Licenses,\nTags, Permits and Stamps","Non-Resident Hunting \nLicenses,\nTags, Permits and Stamps","Total Hunting License, \nTags,Permits & Stamps**","Cost - Resident Hunting \nLicenses,\nTags, Permits and Stamps","Cost - Non-Resident Hunting \nLicenses,\nTags, Permits and Stamps",Gross Cost - Hunting \nLicenses,Type,Name,Abbreviation,Capital,Population (2015),Population (2019),area (square miles)
0,DC,0,0,0,0,$0,$0,$0,Federal District,District of Columbia,DC,NaN,NaN,"705,749",68


In [617]:
# try merging again by deleting the first column "Type"
df2 = df2.drop(['Type'], axis=1)

In [618]:
df2.head(10)

,Name,Abbreviation,Capital,Population (2015),Population (2019),area (square miles)
0,Alabama,AL,Montgomery,NaN,"4,903,185","52,420"
1,Alaska,AK,Juneau,NaN,"731,545","665,384"
2,Arizona,AZ,Phoenix,NaN,"7,278,717","113,990"
3,Arkansas,AR,Little Rock,NaN,"3,017,804","53,179"
4,California,CA,Sacramento,NaN,"39,512,223","163,695"
5,Colorado,CO,Denver,NaN,"5,758,736","104,094"
6,Connecticut,CT,Hartford,NaN,"3,565,278","5,543"
7,Delaware,DE,Dover,NaN,"973,764","2,489"
8,Florida,FL,Tallahassee,NaN,"21,477,737","65,758"
9,Georgia,GA,Atlanta,NaN,"10,617,423","59,425"


In [619]:
merged = df1.join(df2, lsuffix='State', rsuffix='Abbreviation')
# Why doin join work but not merge???

In [620]:
merged.head()

,State,Paid Hunting License \nHolders*,"Resident Hunting Licenses,\nTags, Permits and Stamps","Non-Resident Hunting \nLicenses,\nTags, Permits and Stamps","Total Hunting License, \nTags,Permits & Stamps**","Cost - Resident Hunting \nLicenses,\nTags, Permits and Stamps","Cost - Non-Resident Hunting \nLicenses,\nTags, Permits and Stamps",Gross Cost - Hunting \nLicenses,Name,Abbreviation,Capital,Population (2015),Population (2019),area (square miles)
1,AK,"93,559","423,501","59,235","482,736","$4,859,356","$9,046,715","$13,906,071",Alaska,AK,Juneau,NaN,"731,545","665,384"
2,AL,"452,400","601,683","45,397","647,080","$9,700,295","$6,715,734","$16,416,029",Arizona,AZ,Phoenix,NaN,"7,278,717","113,990"
3,AR,"343,300","349,098","150,728","499,826","$7,851,601","$11,271,653","$19,123,254",Arkansas,AR,Little Rock,NaN,"3,017,804","53,179"
4,AS,0,0,0,0,$0,$0,$0,California,CA,Sacramento,NaN,"39,512,223","163,695"
5,AZ,"302,383","464,607","88,708","553,315","$13,931,397","$5,968,169","$19,899,566",Colorado,CO,Denver,NaN,"5,758,736","104,094"


In [621]:
merged = merged.rename(columns={
    "Paid Hunting License \nHolders*": "license_holders",
    "Population (2019)": "population"
})

In [645]:
# the states with the highest per capita hunting licence holders
# sort values after dividing the number of "Paid hunting licence holders" by "Population (2019)"
# realise division doesn't work because the numbers are in string
# realise the string cannot be converted to integer because of thousand separators
# so saving the file first and reopen with thousands=","

hunting = merged.to_csv("hunting.csv", index=False)

In [649]:
hunting = pd.read_csv("hunting.csv", thousands=',')
hunting.head()

,State,license_holders,"Resident Hunting Licenses,\nTags, Permits and Stamps","Non-Resident Hunting \nLicenses,\nTags, Permits and Stamps","Total Hunting License, \nTags,Permits & Stamps**","Cost - Resident Hunting \nLicenses,\nTags, Permits and Stamps","Cost - Non-Resident Hunting \nLicenses,\nTags, Permits and Stamps",Gross Cost - Hunting \nLicenses,Name,Abbreviation,Capital,Population (2015),population,area (square miles)
0,AK,93559,423501,59235,482736,"$4,859,356","$9,046,715","$13,906,071",Alaska,AK,Juneau,NaN,731545.0,665384.0
1,AL,452400,601683,45397,647080,"$9,700,295","$6,715,734","$16,416,029",Arizona,AZ,Phoenix,NaN,7278717.0,113990.0
2,AR,343300,349098,150728,499826,"$7,851,601","$11,271,653","$19,123,254",Arkansas,AR,Little Rock,NaN,3017804.0,53179.0
3,AS,0,0,0,0,$0,$0,$0,California,CA,Sacramento,NaN,39512223.0,163695.0
4,AZ,302383,464607,88708,553315,"$13,931,397","$5,968,169","$19,899,566",Colorado,CO,Denver,NaN,5758736.0,104094.0


In [653]:
hunting["license_holders"].astype(int)

0        93559
1       452400
2       343300
3            0
4       302383
5       262009
6       296609
7        32052
8            0
9        15619
10      191501
11      724269
12         543
13       10608
14      214398
15      275244
16      293617
17      250481
18      264331
19      319690
20      424918
21       55139
22      116702
23      201370
24      642242
25      545300
26      472993
27          23
28      271281
29      222309
30      609131
31      135724
32      180932
33       55562
34       68767
35      110874
36       89603
37      550868
38      346220
39      512724
40      328323
41      945731
42        7362
43        7514
44      208843
45      213786
46      694458
47     1094753
48      250565
49      254517
50           0
51       62237
52      171940
53      666670
54      205447
55      135228
56    15202669
Name: license_holders, dtype: int64

In [655]:
hunting["population"].astype(float)

0       731545.0
1      7278717.0
2      3017804.0
3     39512223.0
4      5758736.0
5      3565278.0
6       973764.0
7     21477737.0
8     10617423.0
9      1415872.0
10     1787065.0
11    12671821.0
12     6732219.0
13     3155070.0
14     2913314.0
15     4467673.0
16     4648794.0
17     1344212.0
18     6045680.0
19     6892503.0
20     9986857.0
21     5639632.0
22     2976149.0
23     6137428.0
24     1068778.0
25     1934408.0
26     3080156.0
27     1359711.0
28     8882190.0
29     2096829.0
30    19453561.0
31    10488084.0
32      762062.0
33    11689100.0
34     3956971.0
35     4217737.0
36    12801989.0
37     1059361.0
38     5148714.0
39      884659.0
40     6829174.0
41    28995881.0
42     3205958.0
43      623989.0
44     8535519.0
45     7614893.0
46     1792147.0
47     5822434.0
48      578759.0
49      705749.0
50           NaN
51           NaN
52           NaN
53     3193694.0
54           NaN
55           NaN
56           NaN
Name: population, dtype: float6

In [658]:
hunting["per_capita"] = hunting["license_holders"]/hunting["population"] *100000

In [659]:
hunting["per_capita"]

0     12789.233745
1      6215.381090
2     11375.821624
3         0.000000
4      5250.857133
5      7348.907995
6     30460.049868
7       149.233599
8         0.000000
9      1103.136442
10    10715.950455
11     5715.587365
12        8.065691
13      336.220749
14     7359.247922
15     6160.791087
16     6315.982167
17    18634.039869
18     4372.229427
19     4638.227941
20     4254.772047
21      977.705638
22     3921.241846
23     3281.016087
24    60091.244393
25    28189.502938
26    15356.137806
27        1.691536
28     3054.212981
29    10602.152107
30     3131.205644
31     1294.078118
32    23742.425157
33      475.331719
34     1737.869699
35     2628.755657
36      699.914677
37    52000.026431
38     6724.397587
39    57957.246804
40     4807.653166
41     3261.604640
42      229.634948
43     1204.187894
44     2446.752213
45     2807.472147
46    38750.057891
47    18802.325625
48    43293.495220
49    36063.387975
50             NaN
51             NaN
52          

In [662]:
sorted = hunting.sort_values(by="per_capita", ascending=False)

In [666]:
# Top 10 states with highest number of hunting licence holders per 100,000 people
sorted[['Name','per_capita']].head(10)

,Name,per_capita
24,Montana,60091.244393
39,South Dakota,57957.246804
37,Rhode Island[F],52000.026431
48,Wyoming,43293.495220
46,West Virginia,38750.057891
49,District of Columbia,36063.387975
6,Delaware,30460.049868
25,Nebraska,28189.502938
32,North Dakota,23742.425157
53,Puerto Rico,20874.573456


# Not-Camelot questions

You can answer these without using Camelot.

## Federal rules on assault weapons

Download all of the PDFs from the Bureau of Alcohol, Tobacco, Firearms and Explosives's [Rules and Regulations Library](https://www.atf.gov/rules-and-regulations/rules-and-regulations-library). Filter for a list of all PDFs that contain the word `assault weapon` or `assault rifle`.

> If you're having trouble scraping, maybe someone will be kind enough to drop a list of PDF urls in Slack?

In [749]:
import requests
from bs4 import BeautifulSoup

response = requests.get("https://www.atf.gov/rules-and-regulations/rules-and-regulations-library")
doc = BeautifulSoup(response.text)

In [750]:
links = doc.select("a[href$='download']")

In [751]:
# just scraping the first page of the website

urls = []
for link in links:
    url = "https://www.atf.gov" + link['href']
    urls.append(url)
urls

['https://www.atf.gov/firearms/docs/report/2021-firearms-commerce-report/download',
 'https://www.atf.gov/file/169351/download',
 'https://www.atf.gov/file/169346/download',
 'https://www.atf.gov/file/168286/download',
 'https://www.atf.gov/file/168281/download',
 'https://www.atf.gov/file/55251/download',
 'https://www.atf.gov/file/55546/download',
 'https://www.atf.gov/file/55551/download',
 'https://www.atf.gov/file/97816/download',
 'https://www.atf.gov/file/97826/download',
 'https://www.atf.gov/file/97831/download',
 'https://www.atf.gov/file/97841/download',
 'https://www.atf.gov/file/97856/download',
 'https://www.atf.gov/file/97876/download',
 'https://www.atf.gov/file/97881/download',
 'https://www.atf.gov/file/97891/download',
 'https://www.atf.gov/file/97896/download',
 'https://www.atf.gov/file/97901/download',
 'https://www.atf.gov/file/105771/download',
 'https://www.atf.gov/file/124586/download',
 'https://www.atf.gov/file/128221/download',
 'https://www.atf.gov/file/16

In [773]:
# hey! join together all of these urls
# by putting a new line between each of them
file_content = '\n'.join(urls)

# Write all of that content to a file
with open("urls.txt", "w") as f:
    f.write(file_content)

In [777]:
file_content

'https://www.atf.gov/firearms/docs/report/2021-firearms-commerce-report/download\nhttps://www.atf.gov/file/169351/download\nhttps://www.atf.gov/file/169346/download\nhttps://www.atf.gov/file/168286/download\nhttps://www.atf.gov/file/168281/download\nhttps://www.atf.gov/file/55251/download\nhttps://www.atf.gov/file/55546/download\nhttps://www.atf.gov/file/55551/download\nhttps://www.atf.gov/file/97816/download\nhttps://www.atf.gov/file/97826/download\nhttps://www.atf.gov/file/97831/download\nhttps://www.atf.gov/file/97841/download\nhttps://www.atf.gov/file/97856/download\nhttps://www.atf.gov/file/97876/download\nhttps://www.atf.gov/file/97881/download\nhttps://www.atf.gov/file/97891/download\nhttps://www.atf.gov/file/97896/download\nhttps://www.atf.gov/file/97901/download\nhttps://www.atf.gov/file/105771/download\nhttps://www.atf.gov/file/124586/download\nhttps://www.atf.gov/file/128221/download\nhttps://www.atf.gov/file/165811/download\nhttps://www.atf.gov/file/165581/download\nhttps:/

In [ ]:
# not sure how to proceed

## New immigration judge training materials

Extract the text from [this 2020 guide for new immigration judges](2020-training-materials-2a-201-300.pdf) and save it as a file called `training-material.txt`.

> I took this PDF from [a FOIA request](https://www.muckrock.com/foi/united-states-of-america-10/most-recent-new-immigration-judge-training-materials-120125/#comms) – but the unfortunate thing is *I actually had to remove the OCR layer to make it part of this assignment*. By default everything that goes through Muckrock gets all of the text detected!

In [693]:
!pip install pytesseract
import pytesseract


[notice] A new release of pip available: 22.1.2 -> 22.2.1
[notice] To update, run: pip install --upgrade pip


In [694]:
pip install --upgrade pip

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 16.0 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: pip
    Found existing installation: pip 22.1.2
    Uninstalling pip-22.1.2:
      Successfully uninstalled pip-22.1.2
Note: you may need to restart the kernel to use updated packages.


In [696]:
# This doesn't work 
# text = pytesseract.image_to_string('2020-training-materials-2a-201-300.pdf')

In [698]:
# !brew install --cask adoptopenjdk

Running `brew update --auto-update`...
==> Auto-updated Homebrew!
Updated 2 taps (homebrew/core and homebrew/cask).
==> New Formulae
prr
==> New Casks
v2ray-unofficial

You have 3 outdated formulae installed.
You can upgrade them with brew upgrade
or list them with brew outdated.


To re-install adoptopenjdk, run:
  brew reinstall --cask adoptopenjdk


In [699]:
# !pip install tika

In [700]:
import tika
from tika import parser

In [701]:
# also doesn't work
# parsed = parser.from_file("2020-training-materials-2a-201-300.pdf")
# parsed.keys()
# parsed["content"]
# stext = print(parsed["content"])

In [705]:
!pip install pdfplumber
import pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.0/40.0 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.6/142.6 kB 10.9 MB/s eta 0:00:00


In [711]:
# also doesn't work

# a single page
# with pdfplumber.open(r'2020-training-materials-2a-201-300.pdf') as pdf:
    first_page = pdf.pages[-0]
    print(first_page.extract_text())
    
# for every page
# with pdfplumber.open(r'test.pdf') as pdf:
#     for pages in pdf.pages:
#         print(pages.extract_text())

In [712]:
!pip install opencv-python